https://www.unite.ai/asynchronous-llm-api-calls-in-python-a-comprehensive-guide/
- A guide to make asynchronous calls to OpenAI API with rate limiting.
- Backoff is a python library that provides decorators that can used to wrap a function and retry until a specified condition is met.

### Asyncio

- Asyncio is a Python library that provides asynchronous I/O.
    - 协程（coroutine）：使用async def定义的函数
        - 可以在执行过程中暂停和恢复
        - 必须使用await来调用
    - await 关键字
        - 用于暂停协程执行，等待异步操作完成
        - 只能在async函数内使用
    - asyncio.run
        - 执行 async def 定义的事件循环（event loop）
    - `asyncio.gather()`同时处理多个异步任务
- It relieves you of the burden of worrying about **locks and threads** when writing concurrent programs.
- The foundation of asyncio is the idea of **coroutines**, or functions that can be interrupted and then resumed.

- io bound vs. cpu bound
    - I/O 操作指的是程序需要等待外部资源完成其工作的操作。这些外部资源通常比 CPU 慢得多。当程序执行 I/O 操作时，CPU 常常处于空闲等待状态。
        - time.sleep() (同步) 和 asyncio.sleep() (异步) 通常被归类为 I/O 操作，或者更准确地说，它们表现得像 I/O 操作。
        - 调用大型语言模型 (LLM) API 的操作绝对是 I/O 密集型 (I/O Bound) 操作。
            - 网络通信是核心：
                - 当你调用一个 LLM API 时，你的程序需要通过网络将你的请求（例如，prompt、参数等）发送到托管 LLM 模型的远程服务器。
                - 然后，你的程序必须等待远程服务器处理这个请求。这个处理过程可能涉及到模型加载、计算、生成文本等，这部分是在远程服务器上发生的，而不是在你的本地机器上。
                - 最后，远程服务器通过网络将响应（生成的文本、错误信息等）发送回你的程序。
        - 等待时间远大于本地计算时间：
            - 网络延迟 (Latency)：数据在互联网上传输需要时间。
            - 服务器处理时间：LLM 模型本身可能很大，推理计算也需要时间，尤其是在高负载情况下，服务器可能还需要排队处理你的请求。
            - 数据传输时间：请求和响应的数据量也可能影响传输时间。
            - 在整个 API 调用过程中，你的本地程序大部分时间都花在等待网络传输完成和远程服务器响应上。本地 CPU 在这段时间内基本是空闲的，或者只做一些非常轻量级的工作（比如序列化/反序列化数据）。
        - CPU 利用率低（本地）：
            - 当你的代码执行 `response = await client.chat.completions.create(...)` (或者类似的同步调用 `response = client.chat.completions.create(...)`) 时，你的程序会暂停执行，等待网络操作完成。
            - 在这段等待期间，你的本地 CPU 可以被操作系统或事件循环用于执行其他任务（如果是异步调用且有其他任务）或处于空闲状态。
    - CPU 密集型操作指的是程序主要时间消耗在执行计算任务上，CPU 持续高速运转。
    - CPU密集型任务不适合异步编程的原因
        - 异步编程的优势在于I/O等待时的切换

```python
# I/O密集型任务 - 适合异步
async def io_task():
    await asyncio.sleep(1)  # 在等待I/O时，可以切换到其他任务
    return "完成"

# CPU密集型任务 - 不适合异步
async def cpu_task():
    result = 0
    for i in range(1000000000):  # CPU计算，无法切换
        result += i
    return result
```

### OpenAI vs. AsyncOpenAI

- `async def` 方法, 必须用 `await`
- `非阻塞I/O`：当你调用 await client.chat.completions.create(...) 时，它会告诉 asyncio 的事件循环：“我已经发出了一个网络请求，现在需要等待响应。在我等待的时候，你可以去执行其他任务（比如发出另外一个API请求）。”
- `并发执行`：正因为这种“等待时让出控制权”的特性，asyncio.gather(*tasks) 才能实现真正的并发。它会同时启动所有的API请求，然后等待所有请求完成。总耗时约等于最慢的那个请求的耗时，而不是所有请求耗时的总和。

### Asynchronous LLM API Calls

In [ ]:
from dotenv import load_dotenv, find_dotenv
assert load_dotenv(find_dotenv())

```python
import asyncio
import aiohttp
from openai import AsyncOpenAI

async def generate_text(prompt, client):
    response = await client.chat.completions.create(
        model="gpt-4.1-mini-2025-04-14",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content
    
async def main():
    prompts = [
        "Explain quantum computing in simple terms.",
        "Write a haiku about artificial intelligence.",
        "Describe the process of photosynthesis."
    ]
     
    async with AsyncOpenAI() as client:
        tasks = [generate_text(prompt, client) for prompt in prompts]
        results = await asyncio.gather(*tasks)
     
    for prompt, result in zip(prompts, results):
        print(f"Prompt: {prompt}\nResponse: {result}\n")
        
asyncio.run(main())
```

### Batching and Concurrency Control

```python
async def process_batch(batch, client):
    responses = await asyncio.gather(*[
        client.chat.completions.create(
            model="gpt-4.1-mini-2025-04-14",
            messages=[{"role": "user", "content": prompt}]
        ) for prompt in batch
    ])
    return [response.choices[0].message.content for response in responses]
    
async def main():
    prompts = [f"Tell me a fact about number {i}" for i in range(100)]
    batch_size = 10
     
    async with AsyncOpenAI() as client:
        results = []
        for i in range(0, len(prompts), batch_size):
            batch = prompts[i:i+batch_size]
            batch_results = await process_batch(batch, client)
            results.extend(batch_results)
     
    for prompt, result in zip(prompts, results):
        print(f"Prompt: {prompt}\nResponse: {result}\n")
asyncio.run(main())
```

### semaphore

- asyncio.Semaphore：控制并发；
    - 用于限制同时运行的协程数量，从而防止资源过载或满足某些并发限制需求。
- asyncio.as_completed：获取实时结果；
    - asyncio.as_completed 会在每个任务完成时生成一个迭代器，使您能够立即处理完成的任务，而不必等待所有任务结束。

```python
import asyncio
from openai import AsyncOpenAI

async def generate_text(prompt, client, semaphore):
    async with semaphore:
        response = await client.chat.completions.create(
            model="gpt-4.1-mini-2025-04-14",
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content
        
async def main():
    prompts = [f"Tell me a fact about number {i}" for i in range(100)]
    max_concurrent_requests = 5
    semaphore = asyncio.Semaphore(max_concurrent_requests)
     
    async with AsyncOpenAI() as client:
        tasks = [generate_text(prompt, client, semaphore) for prompt in prompts]
        results = await asyncio.gather(*tasks)
     
    for prompt, result in zip(prompts, results):
        print(f"Prompt: {prompt}\nResponse: {result}\n")
asyncio.run(main())
```